## Explorando datos con un DataFrame Spark

In [18]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns

StatementMeta(, 3f94d8ba-300c-4335-90ca-50c3cf2be4b8, 20, Finished, Available, Finished)

In [19]:
df = spark.table('AW_Customer')
display(df.limit(5))

StatementMeta(, 3f94d8ba-300c-4335-90ca-50c3cf2be4b8, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 051ba7ad-92b8-4173-92b7-475f896b09b5)

In [20]:
# Análisis general

# Número de filas y columnas
num_rows = df.count()
num_cols = len(df.columns)
print("Número de filas:", num_rows)
print("Número de columnas:", num_cols)

# Valores nulos por columna
null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
print("\nValores nulos por columna:")
null_counts.show()

# Porcentaje de nulos por columna
null_percentages = df.select([
    (F.count(F.when(F.col(c).isNull(), c)) / num_rows * 100).alias(c)
    for c in df.columns
])
print("Porcentaje de nulos por columna:")
null_percentages.show()

# Tipos de datos
print("\nTipos de datos por columna:")
print(df.dtypes)

# Valores únicos por columna
print("\nValores únicos por columna:")
for c in df.columns:
    print(f"{c}: {df.select(c).distinct().count()}")

# Estadísticas básicas numéricas y de fecha
print("\nDescripción estadística:")
df.describe().show()

StatementMeta(, 3f94d8ba-300c-4335-90ca-50c3cf2be4b8, 22, Finished, Available, Finished)

Número de filas: 847
Número de columnas: 15

Valores nulos por columna:
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+
|CustomerID|NameStyle|Title|FirstName|MiddleName|LastName|Suffix|CompanyName|SalesPerson|EmailAddress|Phone|PasswordHash|PasswordSalt|rowguid|ModifiedDate|
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+
|         0|        0|    7|        0|       343|       0|   799|          0|          0|           0|    0|           0|           0|      0|           0|
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+

Porcentaje de nulos por columna:
+----------+---------+------------------+---------+-----------------+--------+-----------------+-----------+------

In [21]:
# Agrupaciones

df.groupBy("Title").count().orderBy(F.desc("count")).show(truncate=False)
df.groupBy("SalesPerson").count().orderBy(F.desc("count")).show(truncate=False)
print("Compañías únicas:", df.select("CompanyName").distinct().count())

StatementMeta(, 3f94d8ba-300c-4335-90ca-50c3cf2be4b8, 23, Finished, Available, Finished)

+-----+-----+
|Title|count|
+-----+-----+
|Mr.  |490  |
|Ms.  |340  |
|NULL |7    |
|Sr.  |6    |
|Sra. |4    |
+-----+-----+

+------------------------+-----+
|SalesPerson             |count|
+------------------------+-----+
|adventure-works\shu0    |151  |
|adventure-works\jillian0|148  |
|adventure-works\josé1   |142  |
|adventure-works\jae0    |78   |
|adventure-works\garrett1|78   |
|adventure-works\pamela0 |74   |
|adventure-works\david8  |73   |
|adventure-works\linda3  |71   |
|adventure-works\michael9|32   |
+------------------------+-----+

Compañías únicas: 439


In [22]:
# Conversión de fecha y análisis por año

df = df.withColumn("ModifiedDate", F.to_date("ModifiedDate"))
df.select(F.min("ModifiedDate"), F.max("ModifiedDate")).show()

df.groupBy(F.year("ModifiedDate").alias("Year")).count().orderBy("Year").show()

StatementMeta(, 3f94d8ba-300c-4335-90ca-50c3cf2be4b8, 24, Finished, Available, Finished)

+-----------------+-----------------+
|min(ModifiedDate)|max(ModifiedDate)|
+-----------------+-----------------+
|       2005-07-01|       2009-05-16|
+-----------------+-----------------+

+----+-----+
|Year|count|
+----+-----+
|2005|  308|
|2006|  313|
|2007|  210|
|2008|    6|
|2009|   10|
+----+-----+



In [23]:
# Duplicados exactos
duplicates = df.groupBy(df.columns).count().filter("count > 1")
print("Filas duplicadas exactas:")
duplicates.show()

# Duplicados por nombre + empresa
dup_names = df.groupBy("FirstName", "LastName", "CompanyName").count().filter("count > 1")
print("Duplicados por nombre y compañía:")
dup_names.show()

StatementMeta(, 3f94d8ba-300c-4335-90ca-50c3cf2be4b8, 25, Finished, Available, Finished)

Filas duplicadas exactas:
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+-----+
|CustomerID|NameStyle|Title|FirstName|MiddleName|LastName|Suffix|CompanyName|SalesPerson|EmailAddress|Phone|PasswordHash|PasswordSalt|rowguid|ModifiedDate|count|
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+-----+
+----------+---------+-----+---------+----------+--------+------+-----------+-----------+------------+-----+------------+------------+-------+------------+-----+

Duplicados por nombre y compañía:
+---------+----------+--------------------+-----+
|FirstName|  LastName|         CompanyName|count|
+---------+----------+--------------------+-----+
|  William|    Conner|Urban Sports Empo...|    2|
|Cornelius|   Brandon|Initial Bike Company|    2|
|   Andrew|   Cencini|  Sports Merchandis

In [24]:
# Longitud del email
df = df.withColumn("EmailLength", F.length("EmailAddress"))
df.select(F.min("EmailLength"), F.max("EmailLength"), F.avg("EmailLength")).show()


StatementMeta(, 3f94d8ba-300c-4335-90ca-50c3cf2be4b8, 26, Finished, Available, Finished)

+----------------+----------------+------------------+
|min(EmailLength)|max(EmailLength)|  avg(EmailLength)|
+----------------+----------------+------------------+
|              22|              43|26.611570247933884|
+----------------+----------------+------------------+

